# SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation  
- 논문 리뷰

## Abstract  
- SegNet은 Encoder, Decoder, pixel-wise classifier layer로 구성됨  
- Encoder   
    - VGG16의 architecture와 동일  
- Decoder   
    - 낮은 해상도에서 input의 해상도로 매핑  
    - max-pooling indices를 이용해 upsampling 수행  
    - 그래서 학습 파라미터가 필요없음  
- SegNet  
    - FCN 등 다른 알고리즘과 비교해 trade-off에서 좋은 성능을 보여줌  
    - 학습 가능한 파라미터 수가 적으며 SGD를 이용해 end-to-end로 학습  
    - 빠른 속도, 좋은 성능

## Introduction  
- max-pooling, sub sampling을 수행하다보면 coarse한 feature map이 생성됨  
- 이는 해상도를 낮춰 pixel-wise prediction을 하는 segmentation task에는 적합하지 않음  
- 그래서 저해상도에서 input 해상도로 매핑해 pixel-wise prediction을 하기 위해  
- 그리고 정확한 boundary localization을 수행하기 위해 SegNet을 제안함

- SegNet은 다음과 같이 road scene understanding을 목적으로 만들어짐  
    - appearance (도로, 건물), 특히 차도와 인도 분류  
    - shape (자동차, 보행자)  
    - spatial-relationship (context)  
- SegNet의 architecture  
    - Encoder는 VGG16에서 fully connected layer를 제거한 architecture와 동일  
    - Decoder는 Encoder와 대칭적인 모습  
    - 또한 Decoding 과정에서 각각 대응되는 encoder에서 max pooling indices를 사용해 up-sampling을 수행  
    - **max pooling indices의 benefit**
        - 1. boundary 표현을 개선시킴  
        - 2. end-to-end로 학습 가능한 파라미터의 수를 줄일 수 있음  
        - 3. 이러한 형태의 up-sampling은 약간의 수정만으로 FCN과 CRF-RNN과 같은 architecture를 통합할 수 있음  

<img src = "https://younnggsuk.github.io/assets/segnet_a_deep_convolutional_encoder_decoder_architecture_for_image_segmentation/fig_1.png" width=1000>  


- 위 이미지는 SegNet의 성능을 다음 2가지 scene segmentation task에서 평가  
    - 1. CamVid road scene segmentation  
    - 2. SUN RGB-D indoor scene segmentation  
- Pascal VOC dataset은 이미지에 1~2개의 class만 존재하는 경우가 대부분임  
- 그래서 scene understanding task에는 적합하지 않음  

## Architecture  
<img src = "https://younnggsuk.github.io/assets/segnet_a_deep_convolutional_encoder_decoder_architecture_for_image_segmentation/fig_2.png">
  
- Encoder    
    - Encoder는 VGG16에서 fc layer를 제외한 13개의 conv layer로 구성됨  
    - 구체적으로 보면 Conv + Batch Norm + ReLU  
    - 그리고 max-pooling layer with 2x2 window and stride 2 로 구성됨  
    - fully connected layer를 제거해 고해상도를 유지할 수 있음  
    - 학습 파라미터의 수도 134M에서 14.7M까지 줄였음  
    - pre-trained된 VGG16의 weight들을 사용
    
- Decoder  
    - Encoder layer와 대칭적인 구조를 가짐  
    - Encoder에서 max-pooling indices를 받아 up-sampling을 수행  

- Classification layer  
    - Soft-max 활성화 함수  
    - 각 픽셀별로 K class probability를 계산해 출력  
    - 이때 K는 전체 class 갯수  


- 일반적인 Segmentation task에서 Encoder의 레이어가 깊을수록 boundary detail 정보를 잃게 됨  
- 그래서 encoding 과정에서 boundary 정보를 캡쳐하고 저장할 필요가 있음  
- 만약 메모리 자원에 제한이 없다면 FCN이나 U-Net처럼 skip connection을 이용해 decoder의 feature map과 결합하면 좋은 성능을 보여줄 것이다  
- 그러나 메모리는 제한적이므로 본 논문에서는 max-pooling indices 정보만을 저장하는 방식을 사용함  
- 이는 메모리를 덜 쓰는 대신 정확도가 좀 떨어지지만 효율적인 방식임   
- Decoder network는 해당 encoder feature map에서 memorize된 max-pooling indices를 사용해 up-sampling을 수행함

## Decoder Variants  
<img src = "https://younnggsuk.github.io/assets/segnet_a_deep_convolutional_encoder_decoder_architecture_for_image_segmentation/fig_3.png">

- SegNet과 FCN의 구조를 조금씩 변형시켜 비교  
- decoding 과정에서 각 버전이 성능에 어떤 영향을 미치는지 확인  
- **SegNet-Basic**  
    - 4개의 encoder와 4개의 decoder로 이루어짐  
    - max-pooing indices 사용  
    - batch norm 사용  
    - decoder에 ReLU는 사용하지 않음  
    - conv 과정에서 7x7 filter 적용하며 bias 사용하지 않음  
    
- **FCN-Basic**  
    - encoder는 SegNet-Basic과 동일  
    - decoder는 기존 FCN과 동일  
    
- **SegNet-Basic-SingleChannelDecoder**    
    - encoder는 SegNet-Basic과 동일  
    - decoder의 conv filter는 1개만 사용  
    - 따라서 학습 파라미터 수를 줄이고 inference 속도 향상  
    
- **FCN-Basic-NoAddition**  
    - encoder는 SegNet-Basic과 동일  
    - decoder는 skip connection 제외  
    
- **Bilinear-Interpolation**  
    - encoder는 SegNet-Basic과 동일  
    - decoder는 up-sampling에 bilinear-interpolation을 사용  
    
- **SegNet-Basic-EncoderAddition**  
    - encoder는 SegNet-Basic과 동일  
    - decoder는 encoder의 각 레이어에서 64개의 feature map을 더해줌  
    
- **FCN-Basic-NoDimReduction**  
    - encoder는 SegNet-Basic과 동일  
    - decoder는 FCN-Basic에서 encoder의 정보를 더할 때 차원 축소를 하지 않음

## Training  
- **Data Set**  
    - Training : 367, Test : 233  
    - resolution : 360x480  
    - class: 11개 (road, building, cars, pedestrians, signs, poles, side-walk 등)  
- **Weight init** : he initialization  
- **Optimizer** : SGD momentum 0.9  
- **Learning rate** : 0.1  
- **Batch size** : 12  
- **Loss** : Cross entropy loss  
- **Class Imbalance**  
    - median frequency balancing  
    - 이미지에서 pixel의 대부분은 road, sky, building 등의 클래스들이 차지한다  
    - 이들에게는 1보다 작은 weight를 주고  
    - 그렇지 않은 클래스에는 1보다 큰 weight를 줘서 class imbalance를 해결한다

## Analysis  
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FccS6Mt%2FbtqVtaGu2v5%2FnjEdMEAAvHKaTn8ux3uH01%2Fimg.png">

- decoder variants의 성능을 각각 비교하기 위해 3가지 metric을 사용  
    - 1. global accuracy  
        - data set에서 정확하게 분류된 pixel의 percentage  
    
    - 2. class average accuracy  
        - class 예측 정확도의 평균  
        
    - 3. BF score (boundary F1-measure)  
        - boundary에 대한 예측이 중요하므로 mIoU보다 더 적합  
        
- Performance Comparison  
    - up-sampling 과정에서 bilinear-interpolation은 성능이 가장 안좋음  
    - FCN-Basic이나 SegNet-Basic이 더 성능이 좋게 나옴  
    - 따라서 decoder는 학습 가능한 구조를 사용  
    - SegNet-Basic과 FCN-Basic 둘 다 성능이 잘 나오는 듯  
        - SegNet이 max-pooling indices를 사용하기 때문에 메모리 사용량이 더 적음  
        - FCN은 SegNet보다 11배 더 많은 메모리 사용  
        - inference 속도는 FCN이 더 빠름  
        - SegNet의 경우 64개의 feature map을 사용하지만  
        - FCN의 경우 feature map 차원 축소를 하기 때문에 inference 속도가 더 빠름  
        
- 요약  
    - 1. encoder의 feature map을 모두 사용하는 것이 BF score 기준으로 성능이 가장 좋음  
    - 2. inference하는 동안 메모리가 제한되는 경우 차원 축소, max-pooling indices를 통해 성능을 개선시킬 수 있음  
    - 3. decoder가 클수록 성능 향상

## Benchmarking  
- 두 가지 data set으로 SegNet, FCN, DeconvNet 등 다른 모델들과 비교  
    - road scene segmentation  
    - indoor scene segmentation  

### Road Scene Segmentation  
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbM69Cl%2FbtqVe66L386%2F91k5Eie3Hh5Bp7mk93cobk%2Fimg.png">  
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbR1qfm%2FbtqVswXlNtU%2FaCxtfBiXWF7sgt5P2Hwrx1%2Fimg.png">

- SegNet이 가장 성능이 좋음  
    - 작은 object를 표현하고 다른 모델과 비교해 가장 smooth한 boundary를 생성함  

- SegNet, DeepLab-LargeFOV, FCN, DeconvNet을 비교하면  
    - SegNet이 inference에서 가장 적은 GPU 메모리를 사용하고  
    - DeepLab-LargeFOV는 빠르게 동작한다

### SUN RGB-D Indoor Scenes  
<img src = "https://younnggsuk.github.io/assets/segnet_a_deep_convolutional_encoder_decoder_architecture_for_image_segmentation/fig_5.png">  
<img src = "https://younnggsuk.github.io/assets/segnet_a_deep_convolutional_encoder_decoder_architecture_for_image_segmentation/table_4.png">

- indoor scene segmentation에서도 마찬가지로 SegNet이 다른 모델들에 비해 성능이 좋게 나옴

## Conclusion  
- road scene segmentation, indoor scene segmentator에서 효율적인 architecture인 SegNet을 제안함  
- SegNet은 max-pooling indices를 사용해 메모리를 줄이는 동시에 성능도 개선  
- SegNet은 메모리 사용량 측면에서 다른 모델들보다 훨씬 효율적